# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">LIBRARY and SETTINGS</b>

In [1]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, avg, lit
from datetime import datetime, timedelta
import logging



# Carrega variáveis de ambiente
load_dotenv()
s3_endpoint = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")
# -------------------------
# Configuração do Logging
# -------------------------
def setup_logger():
    # Cria o nome do arquivo de log com timestamp
    log_directory = "/opt/notebook/logs/"
    os.makedirs(log_directory, exist_ok=True)  # Garante que o diretório existe
    log_filename = f"goldAgr_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    log_file = os.path.join(log_directory, log_filename)
    
    # Evita múltiplos handlers
    logger = logging.getLogger("minio_silver")
    if logger.handlers:  # Remove handlers existentes
        logger.handlers = []
    
    logger.setLevel(logging.INFO)
    
    # Formato do log
    formatter = logging.Formatter(fmt='{"level": "%(levelname)s", "message": "%(message)s"}')
    
    # Handler para console
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    
    # Handler para arquivo
    file_handler = logging.FileHandler(log_file)
    file_handler.setFormatter(formatter)
    
    # Adiciona ambos handlers
    logger.addHandler(console_handler)
    logger.addHandler(file_handler)
    
    return logger

logger = setup_logger()

def optimize_iceberg_table(spark, table_path):
    """Executa rotinas de otimização para tabela Iceberg com sintaxe compatível"""
    try:
        # 1. Compactação de arquivos de dados (sem comentários no SQL)
        spark.sql(f"""
            CALL local.system.rewrite_data_files(
                table => '{table_path}',
                options => map(
                    'target-file-size-bytes', '67108864',
                    'min-input-files', '5',
                    'min-file-size-bytes', '33554432'
                )
            )
        """)
        logger.info(f"✅ Compactação concluída para {table_path}")

        # 2. Expurgo de snapshots antigos
        retention_days = 7
        cutoff_date = (datetime.now() - timedelta(days=retention_days)).strftime('%Y-%m-%d %H:%M:%S')
        spark.sql(f"""
            CALL local.system.expire_snapshots(
                table => '{table_path}',
                older_than => timestamp '{cutoff_date}',
                retain_last => 3
            )
        """)
        logger.info(f"🗑️ Snapshots antigos removidos para {table_path}")

        # 3. Limpeza de arquivos órfãos
        spark.sql(f"""
            CALL local.system.remove_orphan_files(
                table => '{table_path}',
                older_than => timestamp '{cutoff_date}'
            )
        """)
        logger.info(f"🧹 Arquivos órfãos removidos para {table_path}")

        # 4. Otimização de metadados
        spark.sql(f"CALL local.system.rewrite_manifests('{table_path}')")
        logger.info(f"📦 Manifestos otimizados para {table_path}")

    except Exception as e:
        logger.error(f"❌ Falha na otimização de {table_path}: {str(e)}")
        raise 
# -------------------------
# Configuração do Spark com Otimizações
# -------------------------
def create_spark_session():
    """Cria uma SparkSession otimizada para Iceberg com MinIO"""
    from pyspark import SparkConf
    
    # Configuração inicial para controle de logs
    conf = SparkConf()
    conf.set("spark.logConf", "false")
    conf.set("spark.ui.showConsoleProgress", "false")
    conf.set("spark.driver.extraJavaOptions", "-Dlog4j.configuration=file:/opt/spark/conf/log4j.properties")
    
    # Verifica se os JARs existem
    iceberg_jar = "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar"
    required_jars = [
        iceberg_jar,
        "/opt/spark/jars/hadoop-aws-3.3.4.jar"
    ]
    
    # Configura a SparkSession
    spark = SparkSession.builder \
        .config(conf=conf) \
        .appName("IcebergOptimizedPipeline") \
        .config("spark.jars", ",".join([j for j in required_jars if os.path.exists(j)])) \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.sql.catalog.local.type", "hadoop") \
        .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
        .config("spark.hadoop.fs.s3a.endpoint", s3_endpoint) \
        .config("spark.hadoop.fs.s3a.access.key", s3_access_key) \
        .config("spark.hadoop.fs.s3a.secret.key", s3_secret_key) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.fast.upload", "true") \
        .config("spark.hadoop.fs.s3a.fast.upload.buffer", "disk") \
        .config("spark.hadoop.fs.s3a.connection.maximum", "100") \
        .config("spark.hadoop.fs.s3a.threads.max", "20") \
        .config("spark.sql.catalog.local.default-namespace", "default") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.shuffle.partitions", "4") \
        .config("spark.default.parallelism", "4") \
        .config("spark.sql.iceberg.handle-timestamp-without-timezone", "true") \
        .getOrCreate()
    
    # Configuração adicional de logs
    spark.sparkContext.setLogLevel("ERROR")
    return spark  

# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">FUNCTIONS AND EXECUTION</b>

In [3]:
# Processamento de agregados para análise
# -------------------------
def process_agregados():
    logger.info("🔧 Processando agregados para análise")
    spark = create_spark_session()
    try:
        # Verifica integridade das tabelas de entrada
        for table in ["local.gold.dim_clientes", "local.gold.fato_vendas"]:
            files = spark.sql(f"SELECT * FROM {table}.files").collect()
            for file in files:
                #logger.info(f"Verificando {table}: Arquivo {file['file_path']}, tamanho: {file['file_size_in_bytes']} bytes")
                if file['file_size_in_bytes'] == 0:
                    raise ValueError(f"Arquivo Parquet vazio detectado em {table}: {file['file_path']}")

        # Carrega as tabelas gold
        dim_clientes = spark.table("local.gold.dim_clientes")
        fato_vendas = spark.table("local.gold.fato_vendas")
        
        # 1. Agregado de vendas por cliente
        vendas_por_cliente = fato_vendas.join(
            dim_clientes, 
            "cliente_id", 
            "inner"
        ).groupBy(
            "cliente_id",
            "nome_cliente",
            "ano_venda",
            "mes_venda"
        ).agg(
            sum("total").alias("total_gasto"),
            count("id").alias("quantidade_compras"),
            avg("total").alias("valor_medio_compra")
        ).withColumn("created_at", lit(datetime.now()))
        
        # Cria/Atualiza tabela de agregados por cliente
        spark.sql("""
            CREATE TABLE IF NOT EXISTS local.gold.agregado_vendas_cliente (
                cliente_id string,
                nome_cliente string,
                ano_venda int,
                mes_venda int,
                total_gasto double,
                quantidade_compras long,
                valor_medio_compra double,
                created_at timestamp
            )
            USING iceberg
            PARTITIONED BY (ano_venda, mes_venda)
            TBLPROPERTIES (
                'write.format.default'='parquet',
                'write.parquet.compression-codec'='snappy',
                'write.target-file-size-bytes'='134217728',
                'commit.retry.num-retries'='10'
            )
        """)
        
        # Escreve os dados (sobrescreve partições)
        vendas_por_cliente.writeTo("local.gold.agregado_vendas_cliente").overwritePartitions()
        
        # Validação dos arquivos escritos
        written_files = spark.sql("SELECT * FROM local.gold.agregado_vendas_cliente.files").collect()
        for file in written_files:
            #logger.info(f"Arquivo escrito em agregado_vendas_cliente: {file['file_path']}, tamanho: {file['file_size_in_bytes']} bytes")
            if file['file_size_in_bytes'] == 0:
                raise ValueError(f"Arquivo Parquet vazio detectado: {file['file_path']}")
        
        # 2. Agregado de vendas por categoria
        vendas_por_categoria = fato_vendas.groupBy(
            "categoria",
            "ano_venda",
            "mes_venda"
        ).agg(
            sum("total").alias("total_vendido"),
            count("id").alias("quantidade_vendas"),
            avg("total").alias("valor_medio_venda")
        ).withColumn("created_at", lit(datetime.now()))
        
        # Cria/Atualiza tabela de agregados por categoria
        spark.sql("""
            CREATE TABLE IF NOT EXISTS local.gold.agregado_vendas_categoria (
                categoria string,
                ano_venda int,
                mes_venda int,
                total_vendido double,
                quantidade_vendas long,
                valor_medio_venda double,
                created_at timestamp
            )
            USING iceberg
            PARTITIONED BY (ano_venda, mes_venda)
            TBLPROPERTIES (
                'write.format.default'='parquet',
                'write.parquet.compression-codec'='snappy',
                'write.target-file-size-bytes'='134217728',
                'commit.retry.num-retries'='10'
            )
        """)
        
        # Escreve os dados (sobrescreve partições)
        vendas_por_categoria.writeTo("local.gold.agregado_vendas_categoria").overwritePartitions()
        
        # Validação dos arquivos escritos
        written_files = spark.sql("SELECT * FROM local.gold.agregado_vendas_categoria.files").collect()
        for file in written_files:
            #logger.info(f"Arquivo escrito em agregado_vendas_categoria: {file['file_path']}, tamanho: {file['file_size_in_bytes']} bytes")
            if file['file_size_in_bytes'] == 0:
                raise ValueError(f"Arquivo Parquet vazio detectado: {file['file_path']}")
        
        logger.info("✅ Agregados processados com sucesso!")
        table_path1 = 'local.gold.agregado_vendas_categoria'
        table_path2 = 'local.gold.agregado_vendas_cliente'
        # Otimização pós-escrita
        optimize_iceberg_table(spark, table_path1)
        optimize_iceberg_table(spark, table_path2)
        
    except Exception as e:
        logger.error(f"❌ Erro ao processar agregados: {str(e)}")
        raise

# -------------------------
# Execução principal
# -------------------------
def main():
    try:
        process_agregados()
        logger.info("🚀 Processamento da camada gold concluído com sucesso!")
    except Exception as e:
        logger.error(f"❌ Erro durante o processamento da camada gold: {str(e)}")
    finally:
        print("finally")
        #spark.stop()

if __name__ == "__main__":
    main()

{"level": "INFO", "message": "🔧 Processando agregados para análise"}
{"level": "INFO", "message": "✅ Agregados processados com sucesso!"}
{"level": "INFO", "message": "✅ Compactação concluída para local.gold.agregado_vendas_categoria"}
{"level": "INFO", "message": "🗑️ Snapshots antigos removidos para local.gold.agregado_vendas_categoria"}
{"level": "INFO", "message": "🧹 Arquivos órfãos removidos para local.gold.agregado_vendas_categoria"}
{"level": "INFO", "message": "📦 Manifestos otimizados para local.gold.agregado_vendas_categoria"}
{"level": "INFO", "message": "✅ Compactação concluída para local.gold.agregado_vendas_cliente"}
{"level": "INFO", "message": "🗑️ Snapshots antigos removidos para local.gold.agregado_vendas_cliente"}
{"level": "INFO", "message": "🧹 Arquivos órfãos removidos para local.gold.agregado_vendas_cliente"}
{"level": "INFO", "message": "📦 Manifestos otimizados para local.gold.agregado_vendas_cliente"}
{"level": "INFO", "message": "🚀 Processamento da camada gold con

finally
